# Numeric Stability and Speed of B-Splines
## Context
Splines are piecewise polynomials that are built as the weighted sum of
integer-shifted B-splines, themselves being piecewise polynomials, too.
The weights are called the spline coefficients. Formally, a spline of
degree $n\in{\mathbb{N}}$ is the mapping

$(1)$
$$f:{\mathbb{R}}\rightarrow{\mathbb{R}},
x\mapsto f(x)=\sum_{k\in{\mathbb{Z}}}\,c[k]\,\beta^{n}(x-k),$$
where $c$ are the spline coefficients and $\beta^{n}$ the B-spline basis.
To accurately evalute the spline $f$ at the argument $x$, it is crucial
that the computation of the basis $\beta^{n}$ be stable numerically.
This is easy to achieve for small degrees; when the degree rises,
however, one has to face the fact that the involved polynomials have a
tendency to be made of delicately balancing terms.

Here, we investigate how the numerical stability depends on the way
B-splines are computed. The splinekit library proposes a remarquably
stable strategy that results in the very fast and *constant-speed* evaluation
of B-splines.

### Classic Formula
The classic, productive representation of the $m$th derivative of a
B-spline of degree $n\in{\mathbb{N}}$ and argument $x\in{\mathbb{R}}$ is

$(2)$
$$\frac{{\mathrm{d}}^{m}\beta^{n}(x)}{{\mathrm{d}}x^{m}}=
\sum_{k=0}^{n+1}\,\left(-1\right)^{k}\,{n+1\choose k}\,
\varsigma^{n-m}(x+\frac{n+1}{2}-k).$$
Unfortunately, the term $\left(-1\right)^{k}$ results in contributions that
tend to cancel each other. This spells numerical trouble, even if the
polynomial simple element $\varsigma^{n}(x)=\frac{1}{2\,n!}\,
{\mathrm{sgn}}(x)\,x^{n}$ has the flavor of a well-behaved canonic monomial.
Another issue lies with the growth of the binomial coefficients with respect
to the degree. (To compute a non-differentiated B-spline, simply set $m=0$.)

### De Boor
Other computational recipes have been devised. For instance, the De Boor's
recurrence relation expresses a B-spline of some degree as a weighted
combination of B-splines of lesser degree. As defined in $(2)$, it turns
out that $\beta^{0}(x)=\varsigma^{0}(x+\frac{1}{2})-\varsigma^{0}(x-
\frac{1}{2}).$ Then, it holds for $n\in{\mathbb{N}}+1$ that

$(3)$
$$\beta^{n}(x)=\frac{1}{n}\,\left(\left(x+\frac{n+1}{2}\right)\,
\beta^{n-1}(x+\frac{1}{2})-\left(x-\frac{n+1}{2}\right)\,
\beta^{n-1}(x-\frac{1}{2})\right).$$

### Splinekit
In this library, B-splines of degree $n=0$ are computed as in the classic
or the De Boor's approach. B-splines of positive degree, however, are
computed as the scalar product

$(4)$
$$\beta^{n}(x)={\mathbf{[\![}}\left|x\right|<\frac{n+1}{2}\,
{\mathbf{]\!]}}\,\left(w^{n}[r][\cdot]\right)^{{\mathsf{T}}}\,
{\mathbf{v}}^{n}(\chi),$$
where the notation ${\mathbf{[\![}}\cdot\,{\mathbf{]\!]}}$ is that of
the Iverson bracket. Moreover, $r=\left\lceil\xi\right\rceil\in{\mathbb{Z}}$
for $\xi=\left(\frac{n-1}{2}-x\right)$, and
$\chi=\left(r-\xi\right)\in[0,1)$. Finally,
$\left(w^{n}[r][\cdot]\right)^{{\mathsf{T}}}$ is the
$(r+1)$th row of the B-spline evaluation matrix ${\mathbf{W}}^{n}\in{\mathbb{Q}}^{\left(n+1\right)\times\left(n+1\right)}$
and ${\mathbf{v}}^{n}(\chi)\in{\mathbb{R}}^{n+1}$ is the Vandermonde
vector of argument $\chi$ and degree $n$. In this formulation of a
B-spline, the fact that the Vandermonde vector has the domain $[0,1)$
greatly favors numerical stability since the range of each of its
components is also $[0,1)$.

The matrix ${\mathbf{W}}^{n}$ depends on the degree only and can be
precomputed and cached. Its component at the $(r+1)$th row and $(c+1)$th
column is rational and is defined as

$(5)$
$$w_{r+1,c+1}^{n}=w^{n}[r][c]=\frac{1}{c!}\,
\left(\left.\frac{{\mathrm{d}}^{c}\beta^{n}(x)}{{\mathrm{d}}x^{c}}
\right|_{x=\frac{n-1}{2}-r}+\frac{1}{2}\,
{\mathbf{[\![}}c=n\,{\mathbf{]\!]}}\,\left(-1\right)^{n-r}\,
{n+1\choose r+1}\right).$$
The B-spline derivatives that appear in $(5)$ are computed as
rational numbers, through $(2)$. This leads to a rational value for
$w^{n}[r][c]$, which is then cached as its ``float`` approximation.
With this strategy, the balancing act that was governed in $(2)$ by
the term $\left(-1\right)^{k}$ is now applied to rational values and
can be achieved exactly in the rational domain.

## Goal
Here, we want to compare four computational approaches.
-   In the first (ground-truth) approach, we rely on the observation
    that $\beta^{n}:{\mathbb{R}}\rightarrow{\mathbb{R}}$
    is defined in $(2)$ in such a way that $\beta^{n}$ would map a
    rational number to a number that is rational, too, so that
    $\beta^{n}:{\mathbb{Q}}\rightarrow{\mathbb{Q}}$. This allows us
    to compute exact, ground-truth values by relying on Python's
    built-in ``Fraction`` type.
-   In the second (classic) approach, we compute $\beta^{n}$ as in
    $(2)$, this time relying on Python's built-in ``float`` type.
-   In the third (De Boor) approach, we compute $\beta^{n}$ as in $(3)$,
    again relying on Python's built-in ``float`` type.
-   In the fourth (splinekit) approach, we compute $\beta^{n}$ as in
    $(4)$, relying on Python's built-in ``float`` type.

What we measure is by how far the methods depart from the ground truth,
in a mean-square sense. The error being most noticeable near the end
of the left and right tails of the B-spline, this is where the error
is computed. We also report the computational time.

## Experiment

In [1]:
# Load the required libraries.
from fractions import Fraction
import math
import numpy as np
import time

import splinekit as sk # This library

# Experimental conditions
highest_degree = 16 # Computation time shows a steep increase with the degree
size = 2 * 200 # Number of random arguments
tail_chunk = 2 # Outward support where the computations are performed

# Initialize the generator of random numbers
rng = np.random.default_rng()

# Ground-truth arguments
# The most critical part is near the ends of the tails of the B-spline
random_rational_args = [
    np.concatenate((
        # Support of length 'tail_chunk' near the left end of the tail
        [
            Fraction(x).limit_denominator(2 ** 48)
            for x in rng.uniform(
                low = -0.5 * (degree + 1),
                high = -0.5 * (degree + 1) + tail_chunk,
                size = size // 2
            )
        ],
        # Support of length 'tail_chunk' near the right end of the tail
        [
            Fraction(x).limit_denominator(2 ** 48)
            for x in rng.uniform(
                low = 0.5 * (degree + 1) - tail_chunk,
                high = 0.5 * (degree + 1),
                size = size // 2
            )
        ]
    ))
    for degree in range(highest_degree + 1)
]

# Populate the cache of splinekit
# The cache is persistent throughout the life of the kernel
# This step is done here only to favor a consistency in timings; it is functionally neutral
for degree in range(highest_degree + 1):
    sk.b_spline(0, degree)

# First approach: ground truth
ground_truth = []
for (n, xn) in enumerate(random_rational_args):
    start = time.perf_counter()
    data = [sk.spline_utilities._db_frac(x, n, 0) for x in xn]
    end = time.perf_counter()
    ground_truth += [(data, end - start)]

# Second approach: compute B-splines as in (2) with float accuracy
classic = []
for (n, xn) in enumerate(random_rational_args):
    start = time.perf_counter()
    data = [sk.spline_utilities._db(float(x), n, 0) for x in xn]
    end = time.perf_counter()
    classic += [(data, end - start)]

# De Boor's relation
def de_boor_b_spline (
    x: float,
    n: int
) -> float:
    if 0 == n:
        # Initial condition of the recurrence relation
        return (sk.polynomial_simple_element(x + 0.5, 0) -
            sk.polynomial_simple_element(x - 0.5, 0))
    # Recurrence relation
    return (((x + 0.5 * (n + 1)) * de_boor_b_spline(x + 0.5, n - 1) -
        (x - 0.5 * (n + 1)) * de_boor_b_spline(x - 0.5, n - 1)) / n)

# Third approach: De Boor's recurrence with float accuracy
de_boor = []
for (n, xn) in enumerate(random_rational_args):
    start = time.perf_counter()
    data = [de_boor_b_spline(float(x), n) for x in xn]
    end = time.perf_counter()
    de_boor += [(data, end - start)]

# Fourth approach: splinekit library, with float accuracy
sk_library = []
for (n, xn) in enumerate(random_rational_args):
    start = time.perf_counter()
    data = [sk.b_spline(float(x), n) for x in xn]
    end = time.perf_counter()
    sk_library += [(data, end - start)]

# Mean-square error
def mse (
    gt: [Fraction],
    data: [float]
) -> float:
    error = [
        (float(frac) - data[k]) ** 2
        for (k, frac) in enumerate(gt)
    ]
    return math.fsum(error)

# Signal-to-noise ratio in dB units
def snr_db (
    gt: [Fraction],
    data: [float]
) -> float:
    signal = mse(gt, np.zeros(size, dtype = float))
    noise = mse(gt, data)
    return 10.0 * math.log10(signal / noise) if 0.0 != noise else float("inf")

# Table of results
print()
print("==================================================================================")
print("       |   Ground-Truth   |     Classic      |     De Boor      |    Splinekit    ")
print("Degree | SNR[dB]  Time[s] | SNR[dB]  Time[s] | SNR[dB]  Time[s] | SNR[dB]  Time[s]")
print("-------+------------------+------------------+------------------+-----------------")
for (n, gtn) in enumerate(ground_truth):
    print("    {0:2d} | {1:7.1f}  {2:7.1e} | {3:7.1f}  {4:7.1e} | {5:7.1f}  {6:7.1e} | {7:7.1f}  {8:7.1e}".format(
        n,
        snr_db(gtn[0], ground_truth[n][0]),
        ground_truth[n][1],
        snr_db(gtn[0], classic[n][0]),
        classic[n][1],
        snr_db(gtn[0], de_boor[n][0]),
        de_boor[n][1],
        snr_db(gtn[0], sk_library[n][0]),
        sk_library[n][1]
    ))
print("==================================================================================")


###################
#                 #
#   Be patient!   #
#                 #
###################
# Desktop computer of year 2021: ~20s




       |   Ground-Truth   |     Classic      |     De Boor      |    Splinekit    
Degree | SNR[dB]  Time[s] | SNR[dB]  Time[s] | SNR[dB]  Time[s] | SNR[dB]  Time[s]
-------+------------------+------------------+------------------+-----------------
     0 |     inf  3.2e-03 |     inf  2.5e-04 |     inf  1.6e-04 |     inf  1.3e-04
     1 |     inf  7.2e-03 |     inf  5.5e-04 |     inf  3.2e-04 |     inf  1.3e-03
     2 |     inf  1.1e-02 |   309.6  6.6e-04 |     inf  6.3e-04 |   326.4  1.3e-03
     3 |     inf  1.4e-02 |   296.9  7.8e-04 |   320.2  1.3e-03 |   317.5  1.3e-03
     4 |     inf  1.8e-02 |   282.6  8.9e-04 |   320.6  2.5e-03 |   311.9  1.3e-03
     5 |     inf  2.2e-02 |   261.4  1.0e-03 |   318.7  5.0e-03 |   308.5  1.3e-03
     6 |     inf  2.7e-02 |   243.7  1.2e-03 |   317.7  1.0e-02 |   304.2  1.3e-03
     7 |     inf  3.3e-02 |   224.2  1.3e-03 |   316.9  2.0e-02 |   304.7  1.3e-03
     8 |     inf  3.8e-02 |   199.7  1.4e-03 |   313.4  4.0e-02 |   296.3  1.3e-03
   

## Discussion
-   The numerical stability of the classic approach is consistently the
    worst at all degrees. Moreover, it even collapses when the degree
    rises: for degree $n=16$ already, there is more noise than data.
    The computational cost exhibits a linear increase over the degrees
    explored here. Yet, the classic approach is still the fastest over
    the goldilocks range of degrees $n\in\{3,4,5,6\}$, a range where its
    accuracy can also be considered sufficient for most purposes.
-   The numerical stability of the De Boor's approach is always the best
    (310dB corresponds to about 51 significant bits) but comes at a
    computational cost that increases exponentially with the degree.
    Yet, the trivial code proposed here comes the fastest over the small
    range of degrees $n\in\{0,1,2\}$. For such low degrees, however, one
    has to realize that the computational cost depends more on language
    business (*e.g.*, namespace lookup, recursivity *vs* loops, checks on
    type and validity of the parameters) than on algorithmic considerations.
-   The numerical stability of the splinekit library is consistently
    much higher than that of the classic approach and degrades more
    slowly with the degree (270dB corresponds to about $45$ significant
    bits). Indeed, one has to reach the very high degree $n=94$ before
    the $0$dB threshold is crossed.
-   For degree $n=16$, splinekit is about eight thousand times
    faster than De Boor while, for degree $n=7$ and higher, the splinekit
    strategy is always faster than the other three. The computation time
    of splinekit remains constant through all degrees, a property that
    holds up to $n=94$ and beyond.

## Conclusion
B-splines of small degrees can be computed through the classic method
if one is willing to sacrifice some modest amount of accuracy in the
interest of speed. For higher degrees, however, the approach followed
by the splinekit library is to be preferred. The De Boor's method is not
recommended in reason of its outrageously slow performance.